# Semantic segmentation test

---

**Author:** [rodoart](https://github.com/rodoart/)<br>
**Date created:** 2021/06/24<br>
**Last modified:** 2021/07/15<br>
**Description:** 
Using pretrained neural networks to segment images of rooms.

# Semantic segmentation of rooms

## Path config

If you want the files to be copied to another folder within the same machine you are working on, by a source path other than remote.

In [ ]:
PROJECT_SLUG = 'vigilancia_mascotas'
NAME = 'semantic_segmentation_test_with_room_data'
NUMBER = '0.1'

NOTEBOOK_NAME = f'{NUMBER}-{PROJECT_SLUG}-{NAME}.ipynb'

USING_COLAB = True

if USING_COLAB:
    DRIVE_MOUNT = '/drive'
    REMOTE_PATH = f'{DRIVE_MOUNT}/MyDrive/IA/seminario_innovacion/{PROJECT_SLUG}'
    LOCAL_PATH = '.'
    NOTEBOOK_PATH = f'{DRIVE_MOUNT}/MyDrive/Colab Notebooks/'

else:
    REMOTE_PATH = '..'
    LOCAL_PATH = ''
    DRIVE_MOUNT = ''
    NOTEBOOK_PATH = f'G:\\Mi unidad\\Colab Notebooks\\'

In [ ]:
if USING_COLAB:
    from google.colab import drive
    drive.mount(DRIVE_MOUNT)

Mounted at /drive


## Path functions

In [ ]:
import sys

# It depends on where the library that comes with this package is stored.
sys.path.append(REMOTE_PATH)

In [ ]:
from vigilancia_mascotas.utils.paths import make_two_dir_function, TwoWorkspacePath

In [ ]:
path = make_two_dir_function(
    local_workspace=LOCAL_PATH, 
    remote_workspace=REMOTE_PATH
)


## Prepare the dataset

### Download

For the development of this experiment, the data set of Unity Computer Vision Datasets (Home interior sample) is used, which contains 1,000 artificially created with different types of labeling, including semantic segmentation.

![Example from the dataset](https://content.cdntwrk.com/files/aHViPTEwODAwMiZjbWQ9aXRlbWVkaXRvcmltYWdlJmZpbGVuYW1lPWl0ZW1lZGl0b3JpbWFnZV82MGY5YmY3MGYxNmZkLnBuZyZ2ZXJzaW9uPTAwMDAmc2lnPTIzZTcwMzIwNzIzMGIyMzAwNmZkM2VhNGZiODFiYzkz)

Access to the dataset can be obtained directly on the provided page. The following download section is designed to work only for contributors to this project, but can be easily modified once you get the download link.

In [ ]:
from vigilancia_mascotas.data.make_dataset import DataDownload

In [ ]:
data_object = DataDownload(local_workspace=path().local,
                           remote_workspace=path().remote)
data_object.start()
data_path = data_object.dataset_processed_path

## Pre-trained models selection

In [ ]:
if DRIVE_MOUNT:
    !pip install keras==2.4.3
    !pip install tensorflow==2.4.1
    !pip install keras_segmentation

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220527125636 requires keras<2.9,>=2.8.0rc0, but you have keras 2.4.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394.3 MB 14 kB/s 
     |████████████████████████████████| 132 kB 58.2 MB/s 
     |████████████████████████████████| 14.8 MB 43.6 MB/s 
     |████████████████████████████████| 462 kB 70.7 MB/s 
     |████████████████████████████████| 3.8 MB 45.9 MB/s 
     |████████████████████████████████| 2.9 MB 42.6 MB/s 
  Created w

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 10.1 MB/s 
  Created wheel for keras-segmentation: filename=keras_segmentation-0.3.0-py3-none-any.whl size=29071 sha256=79aca6f326d3e0966e10d90328ce0431adbd6d41e83aa854485fd058e3b6c136
  Stored in directory: /root/.cache/pip/wheels/68/dc/b3/596a3e2461ba16e935ef31661c26e823f841cfb577cec4c47a
Successfully built keras-segmentation
  Attempting uninstall: imageio
    Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
from keras_segmentation.pretrained import pspnet_50_ADE_20K , pspnet_101_cityscapes, pspnet_101_voc12


In [ ]:
val_images_dir = data_path.local.joinpath('val_images')
val_labels_dir = data_path.local.joinpath('val_labels')

In [ ]:
def test_models(models, inp_images_dir,inp_annotations_dir):

  # Verify type of files.
  def change_to_str(dir_to_test):
    if type(dir_to_test)==TwoWorkspacePath:
      return str(inp_images_dir.local)
    else:
      return str(inp_images_dir)
    
  images_dir = change_to_str(inp_images_dir)
  labels_dir = change_to_str(inp_annotations_dir)
    
  # Dictionary to store:  
  all_evaluations = {}

  # Evaluations
  for model_function in models:
    model = model_function()
    eval_dict = model.evaluate_segmentation(inp_images_dir=images_dir, 
                                annotations_dir=labels_dir )
    
    all_evaluations[model_function.__name__] = eval_dict 

  # Change 'class_wise_IU' to list, in order to store as json.
  for key, value in all_evaluations.items():
    all_evaluations[key]['class_wise_IU'] = value['class_wise_IU'].tolist()

  

  # Showing results.
  for key, value in all_evaluations.items():
    print(key)
    for in_key, in_value in value.items():
      if in_key != 'class_wise_IU':
        print(f'{in_key}: {in_value}')
    print('')

  return all_evaluations

In [ ]:
model_functions = [pspnet_50_ADE_20K, pspnet_101_cityscapes, pspnet_101_voc12]

all_evaluations = test_models(
    models=model_functions, 
    inp_images_dir=val_images_dir ,
    inp_annotations_dir=val_labels_dir
  )

187842560/187839752 [==============================] - 1s 0us/step


300it [40:47,  8.16s/it]


264151040/264145248 [==============================] - 3s 0us/step


300it [1:31:59, 18.40s/it]


264151040/264149344 [==============================] - 7s 0us/step


300it [40:56,  8.19s/it]


https://arxiv.org/pdf/1411.4038.pdf

In [ ]:
import json 
from os import makedirs

In [ ]:
json_path = path('data', 'interim', 'first_results.json')

In [ ]:
def upload_dict_as_json(dict_to_upload, json_path):
  if not json_path.local.parent.is_dir():
    makedirs(json_path.local.parent)

  with open(json_path.local, 'w') as fp:
    json.dump(dict_to_upload, fp)

  json_path.upload()
  print('The dictionary has been uploaded.')

In [ ]:
upload_dict_as_json(
    dict_to_upload = all_evaluations, 
    json_path = json_path)

## Test with the default dataset

In [ ]:
val_images_dir_default = path('data','raw','semantic_segmentation', 'dataset1', 'val_images')
val_labels_dir_default = path('data','raw','semantic_segmentation', 'dataset1', 'val_segmentation')
val_images_dir_default.download()
val_labels_dir_default.download()

model_functions = [pspnet_50_ADE_20K, pspnet_101_cityscapes, pspnet_101_voc12]

In [ ]:
all_default_evaluations = test_models(
    models=model_functions, 
    inp_images_dir=val_images_dir_default ,
    inp_annotations_dir=val_labels_dir_default
  )

101it [12:49,  7.62s/it]
101it [30:25, 18.07s/it]
101it [13:41,  8.13s/it]

pspnet_50_ADE_20K
frequency_weighted_IU: 0.0002305893273537063
mean_IU: 0.00037696660702621104

pspnet_101_cityscapes
frequency_weighted_IU: 0.3052844296656404
mean_IU: 0.02002972763702657

pspnet_101_voc12
frequency_weighted_IU: 0.8294396195511148
mean_IU: 0.044063458734799284



In [ ]:
json_default_path = path('data', 'interim', 'default_results.json')


In [ ]:
upload_dict_as_json(
    dict_to_upload = all_default_evaluations, 
    json_path = json_default_path )

The dictionary has been uploaded.


## Label prediction

In [ ]:
from os import listdir

In [ ]:
val_labels_dir = path('data', 'processed', 'semantic_segmentation', 'unity_residential_interiors', 'val_labels')
predicted_val_labels_dir = path( 'data', 'processed', 'semantic_segmentation', 'unity_residential_interiors', 'predicted_val_labels')

makedirs(predicted_val_labels_dir.local, exist_ok=True)


model = pspnet_50_ADE_20K()

In [ ]:
file_names = [file_name for file_name in listdir(val_labels_dir.local) if file_name.endswith('png')]

['863.png',
 '190.png',
 '542.png',
 '1079.png',
 '257.png',
 '80.png',
 '994.png',
 '1044.png',
 '471.png',
 '206.png',
 '300.png',
 '638.png',
 '995.png',
 '247.png',
 '926.png',
 '1023.png',
 '319.png',
 '623.png',
 '555.png',
 '47.png',
 '779.png',
 '428.png',
 '736.png',
 '289.png',
 '633.png',
 '1020.png',
 '576.png',
 '1112.png',
 '475.png',
 '794.png',
 '701.png',
 '1103.png',
 '571.png',
 '333.png',
 '665.png',
 '37.png',
 '95.png',
 '980.png',
 '1104.png',
 '788.png',
 '105.png',
 '949.png',
 '1088.png',
 '917.png',
 '401.png',
 '872.png',
 '858.png',
 '67.png',
 '345.png',
 '802.png',
 '1054.png',
 '618.png',
 '433.png',
 '543.png',
 '1099.png',
 '1097.png',
 '409.png',
 '241.png',
 '536.png',
 '936.png',
 '117.png',
 '942.png',
 '1027.png',
 '244.png',
 '983.png',
 '348.png',
 '311.png',
 '1096.png',
 '1100.png',
 '29.png',
 '265.png',
 '497.png',
 '28.png',
 '16.png',
 '1049.png',
 '604.png',
 '649.png',
 '910.png',
 '431.png',
 '317.png',
 '721.png',
 '74.png',
 '240.png'

In [ ]:
makedirs(predicted_val_labels_dir)

In [ ]:
out = model.predict_segmentation(
    inp="input_image.jpg",
    out_fname="out.png"
)

## Update remote folders

In [ ]:
if USING_COLAB:
    
    TwoWorkspacePath(
        NOTEBOOK_NAME,
        local_workspace=LOCAL_PATH,
        remote_workspace=NOTEBOOK_PATH
        ).upload()

else:
    TwoWorkspacePath(
        NOTEBOOK_NAME,
        local_workspace = make_two_dir_function('notebooks').local,
        remote_workspace=NOTEBOOK_PATH,
        ).upload()
